In [2]:
import requests
import json
import pandas as pd
import datetime
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('njberland', 'wmmhuc7bkj')

SERVERURL = "http://146.185.167.10/resampled/"

datatypes = ["fluorescens", "temp", "salt", "oxygene",  "turbidity"]
datatype = 'temp'

# get the data from the server
r = requests.get(SERVERURL+datatype+".json")

# make the JSON data native Python datastructure
d = json.loads(r.text)

# view one of the dive data
print(d[0])

{'timeframe': '3H', 'ts': {'$date': 1431442800000}, 'divedata': [{'temp': 8.684283080366667, 'pressure(dBAR)': 3.5}, {'temp': nan, 'pressure(dBAR)': 14.5}, {'temp': 8.646019395633333, 'pressure(dBAR)': 6.5}, {'temp': 8.653612691666666, 'pressure(dBAR)': 5.5}, {'temp': 8.668850115933333, 'pressure(dBAR)': 4.5}, {'temp': nan, 'pressure(dBAR)': 18.5}, {'temp': nan, 'pressure(dBAR)': 9.5}, {'temp': 8.7328859405, 'pressure(dBAR)': 2.5}, {'temp': 8.637105864433332, 'pressure(dBAR)': 7.5}, {'temp': nan, 'pressure(dBAR)': 12.5}, {'temp': nan, 'pressure(dBAR)': 15.5}, {'temp': nan, 'pressure(dBAR)': 11.5}, {'temp': nan, 'pressure(dBAR)': 13.5}, {'temp': nan, 'pressure(dBAR)': 17.5}, {'temp': 8.795728878633334, 'pressure(dBAR)': 1.5}, {'temp': 8.9912156023, 'pressure(dBAR)': 0.5}, {'temp': 8.624773825233333, 'pressure(dBAR)': 8.5}, {'temp': nan, 'pressure(dBAR)': 10.5}, {'temp': nan, 'pressure(dBAR)': 16.5}], '_id': {'$oid': '579a4f7b2505660da2e1e2fc'}, 'datatype': 'temp'}


In [8]:
# make a N*M matrix of the data and make sure the number of bins are OK
# we assume that all dives have a measurement that begins at 0.5 - if this is not true we need some rewrite              
y= []
x= []
tempz = []

# create the matrix and lists for x and y labels
for i in d:
    x.append(datetime.datetime.fromtimestamp(i['ts']['$date']/1000))
    
    col = []

    # sort the list so the dictionaries are sorted according to pressure
    newlist = sorted(i['divedata'], key=lambda k: -k['pressure(dBAR)'])
    for j in newlist:
        col.append(j[datatype])
        if -j['pressure(dBAR)'] not in y:
            y.append(-j['pressure(dBAR)'])

    tempz.append(col)
    #x.append(i['ts'])
z = list(map(list, zip(*tempz)))
y = sorted(y)

# tell plotly what to plot!
data=[
    go.Heatmap(
        z=z,
        x=x,
        y=y,
        type='heatmap'
        ),
    ]

# go do it!
py.iplot(data, filename='labelled-heatmap')